In [2]:
import folium
import folium.plugins
import pandas as pd
from folium.features import *
from math import sin, cos, sqrt, atan2, radians

In [112]:
def get_distance(p0, p1):
    R = 6373.0

    lat1 = radians(p0[1])
    lon1 = radians(p0[0])
    lat2 = radians(p1[1])
    lon2 = radians(p1[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [75]:
df_cities = pd.read_csv('./data/worldcitiespop.csv', encoding='latin-1', names=['co','name','reg_name','code','pop','lat','lon'])

/home/jfm/anaconda3/envs/colombiaendato/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [121]:
df_cities_M = df_cities[(df_cities['pop'] > 800000)]
lat_eu_avg = df_cities_M[df_cities_M['name'].isin(['madrid','paris','berlin','moscow'])]['lat'].mean()
lon_eu_avg = df_cities_M[df_cities_M['name'].isin(['madrid','paris','berlin','moscow'])]['lon'].mean()
lat_co_avg = df_cities_M[df_cities_M['co'].isin(['co'])]['lat'].mean()
lon_co_avg = df_cities_M[df_cities_M['co'].isin(['co'])]['lon'].mean()
print(lat_co_avg, lon_co_avg, lat_eu_avg, lon_eu_avg)
diff_lat = lat_eu_avg - lat_co_avg
diff_lon = lon_eu_avg - lon_co_avg

7.138444439999999 -75.29055553999999 49.383888975000005 12.416388825000002


In [122]:
df_cities_M['lon'] = df_cities_M['lon'] - diff_lon + 2.8
df_cities_M['lat'] = df_cities_M['lat'] - diff_lat - 1.5

/home/jfm/anaconda3/envs/colombiaendato/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jfm/anaconda3/envs/colombiaendato/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [123]:
class DivIcon(MacroElement):
    def __init__(self, html='', size=(30,30), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)

In [158]:
m = folium.Map([4.7110, -74.0721],
                tiles='https://korona.geog.uni-heidelberg.de/tiles/roadsg/x={x}&y={y}&z={z}',
                attr='&copy; <a href="http://osm.org/copyright">OpenStreetMap</a> contributors',
                zoom_start=5)

for city in df_cities_M.iterrows():
    #City
    lat = city[1]['lat']
    lon = city[1]['lon']
    name = city[1]['name']
    if name in d_names:
        folium.map.Marker(
            [lat,lon],icon=folium.Icon(icon='star',color='orange')).add_to(m)
        folium.map.Marker(
            [lat,lon],
            icon=DivIcon(
                size=(150,36),
                anchor=(150,0),
                html=spa_name(name),
                style="""
                    font-size:12px;
                    background-color: transparent;
                    border-color: transparent;
                    text-align: right;
                    """
                )
            ).add_to(m)

m
#m.save('/tmp/map.html')

In [149]:
d_names = {
    'turin':'Turín',
    'milan':'Milán',
    'rome':'Roma',
    'naples':'Nápoles',
    'munich':'Munich',
    'prague':'Praga',
    'cologne':'Colonia',
    'hamburg':'Hamburgo',
    'copenhagen':'Copenhague',
    'berlin':'Berlín',
    'vienna':'Viena',
    'brussels':'Bruselas',
    'budapest':'Budapest'
}
spa_name = lambda x:x if (not x in d_names) else d_names[x] 

In [132]:
folium.map.Marker?